In [1]:
import os
import openai
import tiktoken
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from openai import OpenAI

#### Necessary pip installation commands include:
- pip install os
- pip install openai
- pip install pandas
- pip install numpy
- pip install python-dotenv


#### <b>Note</b>: have a <b>.env</b> file already created for accessing API key

In [2]:
# retrieving API Key from OpenAI platform
load_dotenv()
# tokenizer for GPT-3.5 Turbo
tokenizer = tiktoken.get_encoding("cl100k_base")
input_token_cost = ((2.50 / 1000000))
output_token_cost = ((10.00 / 1000000))
REGULAR_TOXIC_DETECTION_API_KEY = os.getenv("REGULAR_TOXIC_DETECTION_API_KEY")

In [3]:
# instantiating client with API key
general = OpenAI(
    api_key=REGULAR_TOXIC_DETECTION_API_KEY
)

In [4]:
# retrieving dataset utilized for evaluation
df = pd.read_csv('../../data/csv/toxic_classification_evaluation_set.csv')
df['Toxic'] = df['Toxic'].apply(lambda x: str(np.int64(x)))
df

,Text,Toxic
0,believe section Polynomials Maximal LFSRs remo...,0
1,best source Dzierzon biography L Brożek Jan Dz...,0
2,First cover current run Batista cover second i...,0
3,Reviewer Hello account granted reviewer userri...,0
4,Either wikipedia letting anyone become adminis...,1
...,...,...
995,didnt say anything fight Niteshift need put ch...,1
996,P4k Wikipedia delete things people worked worl...,1
997,Shii act intimidation concept intimidation mea...,0
998,would also like say masturbate also gay lol,1


In [5]:
# creating container for holding model completions for comparison
completions = pd.DataFrame(columns=['Text', 'Toxic'])
tokens = pd.DataFrame(columns=['Prompt', 'Input', 'Output', 'Total'])

In [6]:
# processing original GPT-4o model across entire dataset
for index, row in df.iterrows():
    completion = general.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system", 
                "content": "Detect whether either is 1 for toxic or 0 for non-toxic"
            },
            {
                "role": "user",
                "content": row['Text']
            }
        ]
    )
    completions.loc[index] = [row['Text'], completion.choices[0].message.content]
    tokens_object = {
        'prompt': len(tokenizer.encode("Detect whether either is 1 for toxic or 0 for non-toxic")),
        'input': len(tokenizer.encode(f"Detect whether either is 1 for toxic or 0 for non-toxic. {row['Text']}")),
        'output': len(tokenizer.encode(completion.choices[0].message.content))
    }
    total_tokens = tokens_object['input'] + tokens_object['output']
    tokens.loc[index] = [tokens_object['prompt'], tokens_object['input'], tokens_object['output'], total_tokens]


In [ ]:
tokens

,Prompt,Input,Output,Total
0,15,52,1,53
1,15,68,1,69
2,15,48,1,49
3,15,122,1,123
4,15,30,1,31
...,...,...,...,...
995,15,40,1,41
996,15,50,1,51
997,15,94,1,95
998,15,25,1,26


In [ ]:
completions

,Text,Toxic
0,believe section Polynomials Maximal LFSRs remo...,0
1,best source Dzierzon biography L Brożek Jan Dz...,0
2,First cover current run Batista cover second i...,0
3,Reviewer Hello account granted reviewer userri...,0
4,Either wikipedia letting anyone become adminis...,1
...,...,...
995,didnt say anything fight Niteshift need put ch...,1
996,P4k Wikipedia delete things people worked worl...,1
997,Shii act intimidation concept intimidation mea...,0
998,would also like say masturbate also gay lol,1


In [ ]:
# comparing results of model to dataset
compare = completions['Toxic'] == df['Toxic']
gptfour_original_accuracy = compare.values.sum() / compare.size

In [ ]:
compare

0      True
1      True
2      True
3      True
4      True
       ... 
995    True
996    True
997    True
998    True
999    True
Name: Toxic, Length: 1000, dtype: bool

In [ ]:
# GPT-4o model accuracy
print(f"Accuracy: {gptfour_original_accuracy * 100:.2f}%")

Accuracy: 85.60%


In [ ]:
# total token input cost: $0.12595
(tokens['Input'].sum()  * input_token_cost)

0.12595

In [ ]:
# total token output cost: $0.01666
(tokens['Output'].sum() * output_token_cost)

0.01666

In [ ]:
# complete total token cost: $0.14261000000000001
(tokens['Input'].sum() * input_token_cost) + (tokens['Output'].sum() * output_token_cost)

0.14261000000000001

In [ ]:
# 52046
tokens['Total'].sum()

52046